## TLDR;




## Imports

In [15]:
import os
from tqdm import tqdm
import sys
from pycaprio import Pycaprio
from pycaprio.mappings import InceptionFormat
from pathlib import Path
sys.path.append('./lib/')
from impresso.helpers.inception import make_inception_client, find_project_by_name, index_project_documents

## Config

In [49]:
data_basedir = "../data/preparation"
tmp_dir = os.path.join(data_basedir, 'tmp')
schema_path = '../data/preparation/TypeSystem.xml'

## Functions

In [24]:
from pycaprio.mappings import InceptionFormat, AnnotationState
from pycaprio.core.exceptions import InceptionBadResponse

def upload_annotations(
    input_path: str,
    document_id,
    client: Pycaprio,
    project_id: int,
    user: str,
    state: AnnotationState = AnnotationState.IN_PROGRESS
):
    with open(input_path) as annotation_file:
        try:
            new_annotation = client.api.create_annotation(
                project_id,
                document_id,
                user,
                annotation_file,
                annotation_format=InceptionFormat.XMI,
                annotation_state=state
            )
        except InceptionBadResponse as e:
            print(e.bad_response.content)
            raise e

## Do the job

In [9]:
os.environ['INCEPTION_HOST'] = "https://inception.dhlab.epfl.ch/prod/"
os.environ['INCEPTION_USERNAME'] = "mromanello"
os.environ['INCEPTION_PASSWORD'] = "5867vifa2v92qpfksj"

In [10]:
inception_client = make_inception_client()

In [13]:
language = 'fr'
target_project_name = f'ajmc-doubleannot-{language}'
target_project = find_project_by_name(inception_client, target_project_name)

In [14]:
target_project

<Project #65: ajmc-doubleannot-fr>

In [21]:
data_basedir = 'data/preparation/doubleannot/fr/kduc/'

annotated_files = [
    os.path.join(data_basedir, file)
    for file in os.listdir(data_basedir)
]

annotated_files

['data/preparation/doubleannot/fr/kduc/lestragdiesdeso00tourgoog_0076.xmi',
 'data/preparation/doubleannot/fr/kduc/lestragdiesdeso00tourgoog_0061.xmi',
 'data/preparation/doubleannot/fr/kduc/lestragdiesdeso00tourgoog_0134.xmi',
 'data/preparation/doubleannot/fr/kduc/lestragdiesdeso00tourgoog_0152.xmi',
 'data/preparation/doubleannot/fr/kduc/lestragdiesdeso00tourgoog_0025.xmi',
 'data/preparation/doubleannot/fr/kduc/lestragdiesdeso00tourgoog_0024.xmi',
 'data/preparation/doubleannot/fr/kduc/lestragdiesdeso00tourgoog_0142.xmi']

In [27]:
users = ["kduc"]
uploaded_docs = []

idx_id2name, idx_name2id = index_project_documents(target_project.project_id, inception_client)

for inp_path in tqdm(annotated_files):
    file = os.path.basename(inp_path)
    
    if file not in idx_name2id:
        continue
    
    inception_document_id = idx_name2id[file]
    inception_project_id = target_project.project_id
    
    for user in users:
        upload_annotations(
            inp_path,
            inception_document_id,
            inception_client,
            inception_project_id,
            user,
            AnnotationState.COMPLETE
        )
        uploaded_docs.append((inception_document_id, file, inp_path))

100%|██████████| 7/7 [00:07<00:00,  1.10s/it]
